<a href="https://colab.research.google.com/github/john-d-noble/Auto-Lending-Startup-Case-Study-Data-Science-Leadership/blob/master/Copy_of_PJT_Defense_Machining%2C_LLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install numpy-financial

In [4]:
"""
REFINED PJT INVESTMENT FINANCIAL MODEL
======================================

This refactored financial model addresses key issues from the code review:
- Fixed interest calculation bug by removing incorrect division by years
- Fixed cash flow calculation using proper principal repayment from amortization schedule
- Added depreciation (straight-line on startup costs) and tax rate (configurable, default 0.0)
- Removed unused imports (matplotlib, seaborn, truncnorm)
- Enhanced documentation with assumptions on margins, depreciation, taxes, and cash flows
- Replaced numpy_financial with manual implementations for compatibility
- Added inline visualizations using matplotlib and seaborn for key financial information

Author: Financial Analysis Team (Refactored)
Date: 2025
Purpose: Production-ready investment decision support for PJT opportunity
"""

import numpy as np
import pandas as pd
import scipy.optimize as optimize
from numpy.random import multivariate_normal
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import logging
from IPython.display import display

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('default')
sns.set_palette("husl")

# Ensure inline plotting
%matplotlib inline

# =============================================================================
# CUSTOM EXCEPTIONS
# =============================================================================

class FinancialModelError(Exception):
    """Base exception for financial model errors."""
    pass

class InvalidConfigurationError(FinancialModelError):
    """Raised when configuration validation fails."""
    pass

class CalculationError(FinancialModelError):
    """Raised when financial calculations fail."""
    pass

class ValidationError(FinancialModelError):
    """Raised when input validation fails."""
    pass

# =============================================================================
# CONFIGURATION MANAGEMENT
# =============================================================================

@dataclass
class MonteCarloParams:
    """Monte Carlo simulation parameters with validation."""
    contract_factor: Dict[str, float] = field(default_factory=lambda: {
        'mean': 1.0, 'std': 0.1, 'low': 0.5, 'high': 1.5
    })
    gross_margin: Dict[str, float] = field(default_factory=lambda: {
        'mean': 0.30, 'std': 0.05, 'low': 0.0, 'high': 1.0
    })
    op_cost_factor: Dict[str, float] = field(default_factory=lambda: {
        'mean': 1.0, 'std': 0.1, 'low': 0.7, 'high': 1.3
    })
    interest_rate: Dict[str, float] = field(default_factory=lambda: {
        'mean': 0.05, 'std': 0.02, 'low': 0.01, 'high': 0.1
    })

    def __post_init__(self):
        """Validate Monte Carlo parameters."""
        for param_name, param_dict in self.__dict__.items():
            if not isinstance(param_dict, dict):
                continue
            required_keys = {'mean', 'std', 'low', 'high'}
            if not required_keys.issubset(param_dict.keys()):
                raise InvalidConfigurationError(
                    f"Monte Carlo parameter '{param_name}' missing required keys: {required_keys}"
                )
            if param_dict['low'] >= param_dict['high']:
                raise InvalidConfigurationError(
                    f"Monte Carlo parameter '{param_name}': low bound must be less than high bound"
                )

@dataclass
class ModelConfiguration:
    """Financial model configuration with comprehensive validation."""
    # Time horizon
    years: int = 5

    # Revenue and profitability
    revenue_base: List[float] = field(default_factory=lambda: [1_500_000, 3_000_000, 5_000_000, 7_000_000, 10_000_000])
    gross_margin_base: float = 0.30
    net_margin_base: List[float] = field(default_factory=lambda: [-0.0333, 0.10, 0.12, 0.15, 0.15])  # Treated as EBITDA margin before dep, interest, taxes

    # Capital structure
    startup_costs: float = 2_500_000
    equity: float = 1_000_000
    loan_amount: float = 1_500_000

    # Financial rates
    interest_rate_base: float = 0.05
    discount_rate: float = 0.08
    growth_rate: float = 0.02
    tax_rate: float = 0.0

    # Monte Carlo
    mc_simulations: int = 10_000
    mc_params: MonteCarloParams = field(default_factory=MonteCarloParams)

    # Valuation parameters
    exit_multiple: float = 7.5
    management_equity_share: float = 0.1
    utilization_rate: float = 0.8

    # Business logic parameters (now configurable)
    max_gross_margin: float = 0.35
    min_utilization_for_max_margin: float = 0.7
    correlation_matrix: List[List[float]] = field(default_factory=lambda: [
        [0.01, -0.005, 0, 0],
        [-0.005, 0.0025, 0, 0],
        [0, 0, 0.01, 0],
        [0, 0, 0, 0.0004]
    ])

    def __post_init__(self):
        """Validate configuration after initialization."""
        self._validate_configuration()

    def _validate_configuration(self):
        """Comprehensive configuration validation."""
        errors = []

        # Validate basic constraints
        if self.years <= 0:
            errors.append("Years must be positive")

        if len(self.revenue_base) != self.years:
            errors.append(f"Revenue projections ({len(self.revenue_base)}) must match years ({self.years})")

        if len(self.net_margin_base) != self.years:
            errors.append(f"Net margin projections ({len(self.net_margin_base)}) must match years ({self.years})")

        # Validate revenue values
        if any(r <= 0 for r in self.revenue_base):
            errors.append("All revenue values must be positive")

        # Validate margins
        if not (0 <= self.gross_margin_base <= 1):
            errors.append("Gross margin must be between 0 and 1")

        if not (0 <= self.max_gross_margin <= 1):
            errors.append("Maximum gross margin must be between 0 and 1")

        if self.max_gross_margin < self.gross_margin_base:
            errors.append("Maximum gross margin must be >= base gross margin")

        # Validate financial inputs
        if self.startup_costs < 0:
            errors.append("Startup costs cannot be negative")

        if self.equity < 0:
            errors.append("Equity cannot be negative")

        if self.loan_amount < 0:
            errors.append("Loan amount cannot be negative")

        if self.startup_costs != (self.equity + self.loan_amount):
            errors.append("Startup costs must equal equity + loan amount")

        # Validate rates
        if not (0 < self.interest_rate_base < 1):
            errors.append("Interest rate must be between 0 and 1")

        if not (0 < self.discount_rate < 1):
            errors.append("Discount rate must be between 0 and 1")

        if self.growth_rate >= self.discount_rate:
            errors.append("Growth rate must be less than discount rate for terminal value calculation")

        if not (0 <= self.tax_rate <= 1):
            errors.append("Tax rate must be between 0 and 1")

        # Validate utilization
        if not (0 < self.utilization_rate <= 1):
            errors.append("Utilization rate must be between 0 and 1")

        # Validate Monte Carlo parameters
        if self.mc_simulations <= 0:
            errors.append("Monte Carlo simulations must be positive")

        # Validate correlation matrix
        if len(self.correlation_matrix) != 4 or any(len(row) != 4 for row in self.correlation_matrix):
            errors.append("Correlation matrix must be 4x4")

        # Validate other parameters
        if self.exit_multiple <= 0:
            errors.append("Exit multiple must be positive")

        if not (0 <= self.management_equity_share <= 1):
            errors.append("Management equity share must be between 0 and 1")

        if errors:
            raise InvalidConfigurationError(f"Configuration validation failed: {'; '.join(errors)}")

# =============================================================================
# FINANCIAL CALCULATIONS CLASS
# =============================================================================

class FinancialCalculations:
    """Centralized financial calculations with proper error handling."""

    @staticmethod
    def calculate_npv(cash_flows: List[float], discount_rate: float) -> float:
        """Calculate Net Present Value with error handling."""
        try:
            if not cash_flows:
                raise ValueError("Cash flows cannot be empty")
            if not (0 < discount_rate < 1):
                raise ValueError("Discount rate must be between 0 and 1")

            npv = sum(cf / (1 + discount_rate) ** t for t, cf in enumerate(cash_flows))

            if np.isnan(npv) or np.isinf(npv):
                raise CalculationError("NPV calculation resulted in invalid value")

            return npv
        except (ValueError, TypeError) as e:
            raise CalculationError(f"NPV calculation failed: {e}")

    @staticmethod
    def calculate_irr(cash_flows: List[float], max_iterations: int = 100) -> Optional[float]:
        """Calculate Internal Rate of Return with proper error handling."""
        try:
            if not cash_flows:
                raise ValueError("Cash flows cannot be empty")
            if len(cash_flows) < 2:
                raise ValueError("At least 2 cash flows required for IRR calculation")

            # Check for valid cash flow pattern (at least one positive and one negative)
            if all(cf >= 0 for cf in cash_flows) or all(cf <= 0 for cf in cash_flows):
                logger.warning("Invalid cash flow pattern for IRR calculation")
                return None

            def npv_func(r):
                return sum(cf / (1 + r) ** t for t, cf in enumerate(cash_flows))

            irr = optimize.brentq(npv_func, -0.99, 10.0)

            if np.isnan(irr) or np.isinf(irr):
                logger.warning("IRR calculation resulted in invalid value")
                return None

            # Check for reasonable IRR values (between -100% and 1000%)
            if irr < -1 or irr > 10:
                logger.warning(f"IRR value {irr:.2%} seems unreasonable")
                return None

            return irr
        except (ValueError, TypeError, RuntimeError) as e:
            logger.error(f"IRR calculation failed: {e}")
            return None

    @staticmethod
    def calculate_loan_payment(principal: float, rate: float, periods: int) -> float:
        """Calculate loan payment with error handling."""
        try:
            if principal <= 0:
                raise ValueError("Principal must be positive")
            if rate < 0:
                raise ValueError("Interest rate cannot be negative")
            if periods <= 0:
                raise ValueError("Number of periods must be positive")

            if rate == 0:
                return principal / periods

            payment = principal * rate * (1 + rate) ** periods / ((1 + rate) ** periods - 1)

            if np.isnan(payment) or np.isinf(payment):
                raise CalculationError("Loan payment calculation resulted in invalid value")

            return payment
        except (ValueError, TypeError) as e:
            raise CalculationError(f"Loan payment calculation failed: {e}")

# =============================================================================
# MAIN FINANCIAL MODEL CLASS
# =============================================================================

class FinancialModel:
    """Main financial model class with comprehensive analysis capabilities.

    Assumptions:
    - The provided net_margin_base is treated as EBITDA margin before depreciation, interest, and taxes.
    - Depreciation is straight-line over the projection years based on startup costs.
    - Taxes are applied on EBT (Earnings Before Tax).
    - Interest is calculated on the remaining principal each year for amortizing loan.
    - Cash flows are Free Cash Flow to Equity (FCFE): Net Profit + Depreciation - Principal Repayment.
    """

    def __init__(self, config: ModelConfiguration):
        """Initialize financial model with configuration."""
        self.config = config
        self.calc = FinancialCalculations()
        self.base_case: Optional[pd.DataFrame] = None
        self.key_metrics: Dict[str, Union[float, List[float]]] = {}
        self.annual_interests: Optional[List[float]] = None
        self.annual_principals: Optional[List[float]] = None
        self.loan_payment: Optional[float] = None
        self.dep: Optional[float] = None

        # Set random seed for reproducibility
        np.random.seed(42)

        logger.info("Financial model initialized successfully")

    def adjust_gross_margin(self, base_margin: float, utilization_rate: float) -> float:
        """Adjust gross margin based on utilization rate with configurable parameters."""
        try:
            if not (0 <= base_margin <= 1):
                raise ValueError("Base margin must be between 0 and 1")
            if not (0 < utilization_rate <= 1):
                raise ValueError("Utilization rate must be between 0 and 1")

            max_margin = self.config.max_gross_margin
            min_util = self.config.min_utilization_for_max_margin

            if utilization_rate <= min_util:
                return base_margin

            # Linear interpolation for utilization above minimum threshold
            adjustment_factor = (utilization_rate - min_util) / (1.0 - min_util)
            adjusted_margin = base_margin + adjustment_factor * (max_margin - base_margin)

            return min(adjusted_margin, max_margin)
        except (ValueError, TypeError) as e:
            raise CalculationError(f"Gross margin adjustment failed: {e}")

    def calculate_break_even_analysis(self) -> List[float]:
        """Calculate break-even revenue for each year with improved logic."""
        try:
            break_even_revenues = []

            # Calculate adjusted gross margin
            gross_margin_adj = self.adjust_gross_margin(
                self.config.gross_margin_base,
                self.config.utilization_rate
            )

            # Calculate fixed costs (startup costs amortized)
            annual_fixed_costs = self.config.startup_costs / self.config.years

            for year in range(self.config.years):
                # Variable costs as percentage of revenue
                variable_cost_rate = 1 - gross_margin_adj

                # Fixed costs include interest and principal for cash break-even
                annual_interest = self.annual_interests[year]
                annual_principal = self.annual_principals[year]
                total_fixed_costs = annual_fixed_costs + annual_interest + annual_principal

                # Break-even: Fixed Costs / Contribution Margin
                contribution_margin = gross_margin_adj

                if contribution_margin > 0:
                    break_even = total_fixed_costs / contribution_margin
                else:
                    break_even = float('inf')

                break_even_revenues.append(break_even)

            return break_even_revenues
        except Exception as e:
            raise CalculationError(f"Break-even analysis failed: {e}")

    def calculate_terminal_value(self, cash_flows: List[float]) -> Dict[str, float]:
        """Calculate terminal value using multiple methods."""
        try:
            if not cash_flows:
                raise ValueError("Cash flows cannot be empty")

            final_cash_flow = cash_flows[-1]

            # Method 1: Perpetuity Growth Model
            if self.config.growth_rate >= self.config.discount_rate:
                raise ValueError("Growth rate must be less than discount rate")

            perpetuity_value = (final_cash_flow * (1 + self.config.growth_rate) /
                                (self.config.discount_rate - self.config.growth_rate))

            # Method 2: Exit Multiple Model
            # Estimated EBITDA = final_cash_flow + loan_payment (since FCF = EBITDA - loan_payment with tax=0, dep added back)
            estimated_ebitda = final_cash_flow + self.loan_payment
            multiple_value = estimated_ebitda * self.config.exit_multiple

            return {
                'Perpetuity': perpetuity_value,
                'Exit Multiple': multiple_value
            }
        except (ValueError, TypeError) as e:
            raise CalculationError(f"Terminal value calculation failed: {e}")

    def calculate_base_case(self) -> pd.DataFrame:
        """Calculate base case financial projections."""
        try:
            logger.info("Calculating base case financial projections...")

            # Calculate adjusted gross margin
            gross_margin_adj = self.adjust_gross_margin(
                self.config.gross_margin_base,
                self.config.utilization_rate
            )

            # Create time series
            years = np.arange(1, self.config.years + 1)

            # Calculate loan payment
            rate = self.config.interest_rate_base
            periods = self.config.years
            principal = self.config.loan_amount
            self.loan_payment = principal * rate * (1 + rate) ** periods / ((1 + rate) ** periods - 1)

            # Calculate amortization schedule
            remaining = self.config.loan_amount
            self.annual_interests = []
            self.annual_principals = []
            for _ in range(self.config.years):
                interest = remaining * self.config.interest_rate_base
                principal_rep = self.loan_payment - interest
                self.annual_interests.append(interest)
                self.annual_principals.append(principal_rep)
                remaining -= principal_rep

            # Handle floating point precision
            if abs(remaining) > 1e-6:
                logger.warning(f"Loan not fully paid due to rounding: remaining {remaining}")
                self.annual_principals[-1] += remaining

            # Depreciation
            self.dep = self.config.startup_costs / self.config.years

            # Calculate financial metrics
            revenue_base = self.config.revenue_base
            gross_profit_base = [rev * gross_margin_adj for rev in revenue_base]
            ebitda_base = [rev * nm for rev, nm in zip(revenue_base, self.config.net_margin_base)]
            operating_expenses_base = [gp - eb for gp, eb in zip(gross_profit_base, ebitda_base)]
            ebit_base = [eb - self.dep for eb in ebitda_base]
            ebt_base = [eb - int_y for eb, int_y in zip(ebit_base, self.annual_interests)]
            net_profit_base = [ebt * (1 - self.config.tax_rate) for ebt in ebt_base]

            # Calculate cash flows
            cash_flows = [-self.config.startup_costs] + [np_val + self.dep - prin_y for np_val, prin_y in zip(net_profit_base, self.annual_principals)]

            # Calculate ROE
            roe = [np_val / self.config.equity if self.config.equity > 0 else 0
                   for np_val in net_profit_base]

            # Create DataFrame
            self.base_case = pd.DataFrame({
                'Year': years,
                'Revenue': revenue_base,
                'Gross Profit': gross_profit_base,
                'Operating Expenses': operating_expenses_base,
                'Net Profit': net_profit_base,
                'Cash Flow': cash_flows[1:],
                'ROE': roe
            })

            # Calculate key metrics
            self.key_metrics = {
                'NPV': self.calc.calculate_npv(cash_flows, self.config.discount_rate),
                'IRR': self.calc.calculate_irr(cash_flows),
                'Loan Payment': self.loan_payment,
                'Break Even Revenues': self.calculate_break_even_analysis(),
                'Terminal Values': self.calculate_terminal_value(cash_flows[1:])
            }

            logger.info("Base case calculation completed successfully")
            return self.base_case

        except Exception as e:
            raise CalculationError(f"Base case calculation failed: {e}")

    def run_sensitivity_analysis(self) -> pd.DataFrame:
        """Run sensitivity analysis with improved error handling."""
        try:
            logger.info("Running sensitivity analysis...")

            if self.base_case is None:
                raise ValueError("Base case must be calculated first")

            # Define sensitivity variables
            variables = {
                'Contract Size': {'base': 1.0, 'low': 0.8, 'high': 1.2},
                'Gross Margin': {'base': self.config.gross_margin_base, 'low': 0.25, 'high': 0.35},
                'Operating Costs': {'base': 1.0, 'low': 0.8, 'high': 1.2},
                'Interest Rate': {'base': self.config.interest_rate_base, 'low': 0.03, 'high': 0.07}
            }

            results = []

            for var_name, params in variables.items():
                for scenario in ['low', 'high']:
                    try:
                        # Create modified configuration
                        modified_config = self._create_modified_config(var_name, params[scenario])

                        # Calculate scenario metrics
                        scenario_results = self._calculate_scenario_metrics(modified_config)

                        results.append({
                            'Variable': var_name,
                            'Scenario': scenario,
                            'Value': params[scenario],
                            'Year 5 Net Profit': scenario_results['net_profit_y5'],
                            'NPV': scenario_results['npv'],
                            'IRR': scenario_results['irr']
                        })

                    except Exception as e:
                        logger.error(f"Sensitivity analysis failed for {var_name} {scenario}: {e}")
                        results.append({
                            'Variable': var_name,
                            'Scenario': scenario,
                            'Value': params[scenario],
                            'Year 5 Net Profit': np.nan,
                            'NPV': np.nan,
                            'IRR': np.nan
                        })

            return pd.DataFrame(results)

        except Exception as e:
            raise CalculationError(f"Sensitivity analysis failed: {e}")

    def _create_modified_config(self, variable: str, value: float) -> ModelConfiguration:
        """Create modified configuration for sensitivity analysis."""
        # Create a copy of the current configuration
        config_dict = self.config.__dict__.copy()

        if variable == 'Contract Size':
            config_dict['revenue_base'] = [r * value for r in self.config.revenue_base]
        elif variable == 'Gross Margin':
            config_dict['gross_margin_base'] = value
        elif variable == 'Interest Rate':
            config_dict['interest_rate_base'] = value
        # Operating costs handled in scenario calculation

        return ModelConfiguration(**config_dict)

    def _calculate_scenario_metrics(self, config: ModelConfiguration) -> Dict[str, float]:
        """Calculate metrics for a scenario configuration."""
        try:
            # Create temporary model with modified config
            temp_model = FinancialModel(config)
            temp_base_case = temp_model.calculate_base_case()

            return {
                'net_profit_y5': temp_base_case['Net Profit'].iloc[-1],
                'npv': temp_model.key_metrics['NPV'],
                'irr': temp_model.key_metrics['IRR'] or np.nan
            }
        except Exception as e:
            logger.error(f"Scenario calculation failed: {e}")
            return {'net_profit_y5': np.nan, 'npv': np.nan, 'irr': np.nan}

    def run_monte_carlo_simulation(self) -> List[float]:
        """Run Monte Carlo simulation with vectorized calculations."""
        try:
            logger.info(f"Running Monte Carlo simulation with {self.config.mc_simulations:,} iterations...")

            # Prepare correlation matrix
            correlation_matrix = np.array(self.config.correlation_matrix)

            # Validate correlation matrix
            if not np.allclose(correlation_matrix, correlation_matrix.T):
                raise ValueError("Correlation matrix must be symmetric")

            # Mean values for multivariate normal distribution
            mean = [
                self.config.mc_params.contract_factor['mean'],
                self.config.mc_params.gross_margin['mean'],
                self.config.mc_params.op_cost_factor['mean'],
                self.config.mc_params.interest_rate['mean']
            ]

            # Generate correlated random variables
            simulations = multivariate_normal(mean, correlation_matrix, self.config.mc_simulations)

            # Apply bounds vectorized
            contract_factors = np.clip(
                simulations[:, 0],
                self.config.mc_params.contract_factor['low'],
                self.config.mc_params.contract_factor['high']
            )

            gross_margins = np.clip(
                simulations[:, 1],
                self.config.mc_params.gross_margin['low'],
                self.config.mc_params.gross_margin['high']
            )

            op_cost_factors = np.clip(
                simulations[:, 2],
                self.config.mc_params.op_cost_factor['low'],
                self.config.mc_params.op_cost_factor['high'] # Corrected typo here
            )

            interest_rates = np.clip(
                simulations[:, 3],
                self.config.mc_params.interest_rate['low'],
                self.config.mc_params.interest_rate['high']
            )

            # Vectorized calculations
            base_revenue = np.array(self.config.revenue_base)
            revenues = base_revenue * contract_factors[:, np.newaxis]
            gross_profits = revenues * gross_margins[:, np.newaxis]

            # Calculate base operating expenses (before dep, interest)
            base_ebitda = np.array(
                [r * nm for r, nm in zip(self.config.revenue_base, self.config.net_margin_base)]
            )
            base_gross = base_revenue * self.config.gross_margin_base
            base_op_expenses = base_gross - base_ebitda

            op_expenses = base_op_expenses * op_cost_factors[:, np.newaxis]

            ebitda = gross_profits - op_expenses

            dep = self.config.startup_costs / self.config.years

            ebit = ebitda - dep

            # Calculate amortization for each simulation
            periods = self.config.years
            principal = self.config.loan_amount
            loan_payments = principal * interest_rates * np.power(1 + interest_rates, periods) / (np.power(1 + interest_rates, periods) - 1)

            remaining = self.config.years
            remaining = np.full(self.config.mc_simulations, self.config.loan_amount, dtype=np.float64)
            net_profits = np.zeros((self.config.mc_simulations, self.config.years))
            for year in range(self.config.years):
                interest = remaining * interest_rates
                principal_rep = loan_payments - interest
                remaining -= principal_rep
                ebt = ebit[:, year] - interest
                net = ebt * (1 - self.config.tax_rate)
                net_profits[:, year] = net

            # Return Year 5 net profit for each simulation
            return net_profits[:, -1].tolist()

        except Exception as e:
            raise CalculationError(f"Monte Carlo simulation failed: {e}")

    def generate_report(self) -> Dict[str, any]:
        """Generate comprehensive financial report."""
        try:
            if self.base_case is None:
                raise ValueError("Base case must be calculated first")

            # Run all analyses
            sensitivity_results = self.run_sensitivity_analysis()
            monte_carlo_results = self.run_monte_carlo_simulation()

            # Compile comprehensive report
            report = {
                'base_case': self.base_case,
                'key_metrics': self.key_metrics,
                'sensitivity_analysis': sensitivity_results,
                'monte_carlo_results': {
                    'data': monte_carlo_results,
                    'statistics': {
                        'mean': np.mean(monte_carlo_results),
                        'median': np.median(monte_carlo_results),
                        'std': np.std(monte_carlo_results),
                        'min': np.min(monte_carlo_results),
                        'max': np.max(monte_carlo_results),
                        'percentiles': {
                            '5th': np.percentile(monte_carlo_results, 5),
                            '25th': np.percentile(monte_carlo_results, 25),
                            '75th': np.percentile(monte_carlo_results, 75),
                            '95th': np.percentile(monte_carlo_results, 95)
                        }
                    }
                },
                'configuration': self.config
            }

            logger.info("Financial report generated successfully")
            return report

        except Exception as e:
            raise CalculationError(f"Report generation failed: {e}")

# =============================================================================
# USAGE EXAMPLE
# =============================================================================

def main():
    """Example usage of the improved financial model."""
    try:
        # Create configuration
        config = ModelConfiguration(
            years=5,
            revenue_base=[1_500_000, 3_000_000, 5_000_000, 7_000_000, 10_000_000],
            gross_margin_base=0.30,
            net_margin_base=[-0.0333, 0.10, 0.12, 0.15, 0.15],
            startup_costs=2_500_000,
            equity=1_000_000,
            loan_amount=1_500_000,
            interest_rate_base=0.05,
            discount_rate=0.08,
            growth_rate=0.02,
            mc_simulations=10_000,
            tax_rate=0.0  # Set to 0.25 for taxes; 0.0 to match original
        )

        # Create and run model
        model = FinancialModel(config)
        base_case = model.calculate_base_case()

        # Generate comprehensive report
        report = model.generate_report()

        # Display results
        print("\n=== BASE CASE RESULTS ===")
        print(base_case)

        # Visualize base case projections
        print("\n=== BASE CASE VISUALIZATION ===")
        fig, ax = plt.subplots(figsize=(10, 6))
        base_case.plot(x='Year', y=['Revenue', 'Net Profit', 'Cash Flow'], kind='line', marker='o', ax=ax)
        ax.set_title('Base Case Financial Projections')
        ax.set_ylabel('Amount ($)')
        ax.grid(True)
        display(fig)

        print("\n=== KEY METRICS ===")
        print(f"NPV: ${report['key_metrics']['NPV']:,.2f}")
        irr = report['key_metrics']['IRR']
        print(f"IRR: {irr:.2%}" if irr is not None else "IRR: Unable to calculate")

        # Visualize key metrics (NPV and IRR)
        print("\n=== KEY METRICS VISUALIZATION ===")
        fig, ax = plt.subplots(figsize=(6, 4))
        metrics_df = pd.DataFrame({
            'Metric': ['NPV', 'IRR'],
            'Value': [report['key_metrics']['NPV'], irr if irr is not None else 0]
        })
        sns.barplot(data=metrics_df, x='Metric', y='Value', ax=ax)
        ax.set_title('Key Financial Metrics')
        ax.set_ylabel('Value')
        if irr is None:
            ax.text(1, 0, 'IRR: Unable to calculate', ha='center', va='bottom')
        display(fig)

        print("\n=== MONTE CARLO STATISTICS ===")
        mc_stats = report['monte_carlo_results']['statistics']
        print(f"Mean Year 5 Net Profit: ${mc_stats['mean']:,.2f}")
        print(f"Median Year 5 Net Profit: ${mc_stats['median']:,.2f}")
        print(f"Standard Deviation: ${mc_stats['std']:,.2f}")
        print(f"5th Percentile: ${mc_stats['percentiles']['5th']:,.2f}")
        print(f"95th Percentile: ${mc_stats['percentiles']['95th']:,.2f}")

        # Visualize Monte Carlo results (histogram)
        print("\n=== MONTE CARLO VISUALIZATION ===")
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(report['monte_carlo_results']['data'], bins=50, kde=True, ax=ax)
        ax.set_title('Distribution of Year 5 Net Profit (Monte Carlo Simulations)')
        ax.set_xlabel('Net Profit ($)')
        ax.set_ylabel('Frequency')
        display(fig)

        # Visualize sensitivity analysis (tornado chart for NPV)
        print("\n=== SENSITIVITY ANALYSIS VISUALIZATION ===")
        sensitivity = report['sensitivity_analysis']
        sensitivity_pivot = sensitivity.pivot(index='Variable', columns='Scenario', values='NPV')
        sensitivity_pivot['Base'] = report['key_metrics']['NPV']
        sensitivity_pivot['Low Impact'] = sensitivity_pivot['low'] - sensitivity_pivot['Base']
        sensitivity_pivot['High Impact'] = sensitivity_pivot['high'] - sensitivity_pivot['Base']
        fig, ax = plt.subplots(figsize=(10, 6))
        sensitivity_pivot[['Low Impact', 'High Impact']].plot(kind='barh', ax=ax)
        ax.set_title('Sensitivity Analysis - Impact on NPV')
        ax.set_xlabel('Change in NPV ($)')
        ax.grid(True)
        display(fig)

        logger.info("Financial model execution completed successfully")

    except Exception as e:
        logger.error(f"Model execution failed: {e}")
        raise